#### Importing libraries

In [1]:
from keras_model import build_model
from env_wrapper import RLlibEnvWrapper
from tf_models import KerasConvLSTM, get_flat_obs_size
import tensorflow as tf

Inside covid19_components.py: 1 GPUs are available.
Inside covid19_env.py: 1 GPUs are available.


/home/ettore/miniconda3/envs/aie-clear/lib/python3.7/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


#### Model and env config

In [2]:
model_config = {
    'custom_model': "keras_conv_lstm",
    'custom_options': {
        'fc_dim': 128,
        'idx_emb_dim': 4,
        'input_emb_vocab': 100,
        'lstm_cell_size': 128,
        'num_conv': 2,
        'num_fc': 2,
    },
    'max_seq_len': 25,

}

env_config = {
    'env_config_dict': {
        'scenario_name': 'layout_from_file/simple_wood_and_stone',
        'components': [
            ('Build', {'skill_dist': "pareto",
             'payment_max_skill_multiplier': 3}),
            ('ContinuousDoubleAuction', {'max_num_orders': 5}),
            ('Gather', {}),
        ],
        'env_layout_file': 'quadrant_25x25_20each_30clump.txt',
        'starting_agent_coin': 10,
        'fixed_four_skill_and_loc': True,
        'n_agents': 2,          # Number of non-planner agents (must be > 1)
        'world_size': [25, 25],  # [Height, Width] of the env world
        'episode_length': 1000,  # Number of timesteps per episode
        'multi_action_mode_agents': False,
        'multi_action_mode_planner': True,
        'flatten_observations': False,
        'flatten_masks': True,
    }
}


/home/ettore/miniconda3/envs/aie-clear/lib/python3.7/site-packages/ipykernel-5.5.5-py3.7.egg/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Building model and env

In [3]:
env = RLlibEnvWrapper(env_config)
obs = env.reset()

model = KerasConvLSTM(env.observation_space, env.action_space, 50, model_config, None)
state = model.get_initial_state()

# obs = {'obs': obs['0']}
# rank_1_tensor = tf.constant([50])
# flat_obs_space = get_flat_obs_size(env.observation_space)


/home/ettore/miniconda3/envs/aie-clear/lib/python3.7/site-packages/ai_economist-1.7.1-py3.7.egg/ai_economist/foundation/scenarios/simple_wood_and_stone/layout_from_file.py:212: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ).astype(np.int)



probabilmente devo prendere i dati da obs[x] e farli diventare un tensore ->
dizionario come dati, ma composto di tensori, quindi in 
`obs`: obs... -> obs... e' un dizionario con chiavi i nomi dei dati e contenuto un tensore.

output, new_state = model.forward(dati, state, rank_1_tensor)

Call the model with the given input tensors and state.

Any complex observations (dicts, tuples, etc.) will be unpacked by
__call__ before being passed to forward(). To access the flattened
observation tensor, refer to input_dict["obs_flat"].

This method can be called any number of times. In eager execution,
each call to forward() will eagerly evaluate the model. In symbolic
execution, each call to forward creates a computation graph that
operates over the variables of this model (i.e., shares weights).

Custom models should override this instead of __call__.

Arguments:
    inputs (dict): observation tensor with shape [B, T, obs_size].
    input_dict (dict): dictionary of input tensors, including "obs",
        "obs_flat", "prev_action", "prev_reward", "is_training"
        input_dict (dict): obs.
    state (list): list of state tensors, each with shape [B, T, size].
    seq_lens (Tensor): 1d tensor holding input sequence lengths.

Returns:
    (outputs, new_state): The model output tensor of shape
        [B, T, num_outputs] and the list of new state tensors each with
        shape [B, size].


from /python3.7/site-packages/ray/rllib/models/tf/tf_modelv2.py

> `input_dict` (dict): dictionary of input tensors, including `"obs", "obs_flat", "prev_action", "prev_reward", "is_training"`

So we want to build this dictionary.

In [20]:
# obs
def dict_to_tensor_dict(a_dict: dict):
    tensor_dict = {}
    for key, value in a_dict.items():
        tensor_dict[key] = tf.convert_to_tensor(value)
    
    return tensor_dict
    
# prova:
obs_tensor_dict = dict_to_tensor_dict(obs['0'])
# "obs", "obs_flat", "prev_action", "prev_reward", "is_training"
input_dict = {
    'obs': obs_tensor_dict,
    # 'obs_flat': ,
    'prev_action': None, 
    'prev_reward': None, 
    'is_training': True
}


/home/ettore/miniconda3/envs/aie-clear/lib/python3.7/site-packages/ipykernel-5.5.5-py3.7.egg/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
rank_1_tensor = tf.constant([50])
output, new_state = model.forward(input_dict, state, rank_1_tensor)


Tensor("Const_115:0", shape=(1,), dtype=float64)
Tensor("Const_116:0", shape=(1,), dtype=float64)
Tensor("Const_119:0", shape=(11,), dtype=float64)
Tensor("Const_125:0", shape=(11,), dtype=float64)
Tensor("Const_120:0", shape=(11,), dtype=float64)
Tensor("Const_126:0", shape=(11,), dtype=float64)
Tensor("Const_117:0", shape=(1,), dtype=float64)
Tensor("Const_123:0", shape=(1,), dtype=float64)
Tensor("Const_121:0", shape=(11,), dtype=float64)
Tensor("Const_127:0", shape=(11,), dtype=float64)
Tensor("Const_122:0", shape=(11,), dtype=float64)
Tensor("Const_128:0", shape=(11,), dtype=float64)
Tensor("Const_118:0", shape=(11,), dtype=float64)
Tensor("Const_124:0", shape=(11,), dtype=float64)
Tensor("Const_129:0", shape=(1,), dtype=float64)
Tensor("Const_130:0", shape=(50,), dtype=float32)
Tensor("Const_114:0", shape=(1,), dtype=float64)
Tensor("Const_108:0", shape=(2, 11, 11), dtype=int16)
Tensor("Const_111:0", shape=(1,), dtype=float64)
Tensor("Const_112:0", shape=(1,), dtype=float64)
Tens

ValueError: Input 0 of layer "permute_1" is incompatible with the layer: expected ndim=5, found ndim=4. Full shape received: (Dimension(1), Dimension(2), Dimension(11), Dimension(11))